# Results Benchmarking

This notebook compares the time and space required to save results files in different formats.

In [ ]:
import os
import tempfile
import timeit

from kbmod.fake_data.fake_result_creator import (
    add_fake_psi_phi_to_results,
    add_fake_coadds_to_results,
    make_fake_in_image_trajectory_info,
    make_fake_results,
    make_fake_trajectories,
)
from kbmod.results import Results

We create a fake set of results to use for benchmarking.  Add the psi/phi curves and three stamps.

In [ ]:
num_times = 20
num_results = 200
img_height = 4000
img_width = 4000
stamp_radius = 25

results = make_fake_results(num_times, img_height, img_width, num_results)
results = add_fake_psi_phi_to_results(results)
for stamp in ["mean", "median", "sum"]:
    results = add_fake_coadds_to_results(results, stamp, stamp_radius)

Create a temporary directory for the tests. Write out and stat files in different formats.

In [ ]:
tmp_dir = tempfile.TemporaryDirectory()

fmt_to_file = {
    "ecsv": os.path.join(tmp_dir.name, "results.ecsv"),
    "parquet": os.path.join(tmp_dir.name, "results.parquet"),
    "hdf5": os.path.join(tmp_dir.name, "results.hdf5"),
}

for fmt, file_name in fmt_to_file.items():
    results.write_table(file_name, overwrite=True)

    file_size_mb = os.path.getsize(file_name) / (1024.0 * 1024.0)
    print(f"{fmt}: {file_size_mb:0.2f} MB")

Time the writing of files in each format.

In [ ]:
%%timeit
results.write_table(fmt_to_file["ecsv"], overwrite=True)

In [ ]:
%%timeit
results.write_table(fmt_to_file["parquet"], overwrite=True)

In [ ]:
%%timeit
results.write_table(fmt_to_file["hdf5"], overwrite=True)

Time the reading of the files. All of these files should exist from the tests above.

In [ ]:
%%timeit
_ = Results.read_table(fmt_to_file["ecsv"], track_filtered=False)

In [ ]:
%%timeit
_ = Results.read_table(fmt_to_file["parquet"], track_filtered=False)

In [ ]:
%%timeit
_ = Results.read_table(fmt_to_file["hdf5"], track_filtered=False)

Clean up the temporary directory and files.

In [ ]:
tmp_dir.cleanup()